In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error
import warnings
import matplotlib.pyplot as plt
import seaborn as sns


def identify_best_column_with_data(df, target_station):
    """
    Identifica la mejor columna que tiene la menor cantidad de valores nulos
    en las filas donde la estación objetivo tiene valores faltantes.

    Parameters:
    - df: DataFrame que contiene las estaciones y los datos de las mismas.
    - target_station: La estación que tiene valores faltantes y que se quiere completar.

    Returns:
    - best_column: Nombre de la columna con la menor cantidad de valores nulos en las filas donde la estación objetivo tiene valores faltantes.
    """
    # Filtrar las filas donde la estación objetivo tiene valores NaN
    target_nan_df = df[df[target_station].isna()]

    # Verificar las columnas que tienen la menor cantidad de NaN en esas mismas filas
    missing_counts = target_nan_df.drop(columns=[target_station]).isna().sum()

    # Seleccionar la columna con la menor cantidad de NaN
    best_column = missing_counts.idxmin()

    return best_column

def simple_regression(df, best_column, target_station):
    # Filtrar las filas donde target_station tiene valores NaN
    target_nan_df = df[df[target_station].isna()]

    # Filtrar las filas donde best_column no tiene valores NaN (tiene datos)
    target_nan_df_non_nan_other_columns_df = target_nan_df[target_nan_df[best_column].notna()]

    # Filtrar las filas completas (sin NaN) en 'best_column' y 'target_station' para entrenar el modelo
    remaining_df = df[['Fecha', best_column, target_station]].dropna()

    # Verificar si hay suficientes datos para el train-test split
    if len(remaining_df) < 2:
        print(f"No hay suficientes datos para realizar regresión en la estación: {target_station}")
        return df

    # Realizar Train-test split (70% train, 30% test)
    train_df, test_df = train_test_split(remaining_df, test_size=0.3, random_state=42)

    # Inicializar el modelo de regresión lineal
    linear_reg = LinearRegression()

    # Entrenar el modelo
    X_train = train_df[[best_column]]  # Usar solo 'best_column' como predictor
    y_train = train_df[target_station]
    linear_reg.fit(X_train, y_train)

    # Usar el modelo entrenado para completar los valores faltantes en target_station
    for index, row in target_nan_df_non_nan_other_columns_df.iterrows():
        # Crear un DataFrame con el valor de 'best_column' para mantener los nombres de las características
        X_pred_df = pd.DataFrame([[row[best_column]]], columns=[best_column])
        predicted_value = linear_reg.predict(X_pred_df)[0]

        # Verificar si el valor predicho es negativo o cercano a 0
        if predicted_value < 0 or np.isclose(predicted_value, 0):
            predicted_value = 0

        # Rellenar el valor predicho en target_station
        target_nan_df_non_nan_other_columns_df.loc[index, target_station] = np.round(predicted_value, 2)

    # Fusionar los datos completados con el resto del DataFrame
    merged_df = pd.concat([target_nan_df_non_nan_other_columns_df, remaining_df], ignore_index=True)
    merged_df.sort_values(by='Fecha', inplace=True)
    
    return merged_df

# Function to perform train-test split and complete missing data using regression
def complete_missing_data(df, target_station):
    # Drop rows with NaN values in the target station
    # Create a copy of the dropped rows
    dropped_df = df[df.isna().any(axis=1)]

    # Filtrar las filas donde target_station es NaN
    target_nan_df = df[df[target_station].isna()]

    # Filtrar las filas donde el resto de las columnas no son NaN
    target_nan_df_non_nan_other_columns_df = target_nan_df[target_nan_df.drop(columns=[target_station]).notna().all(axis=1)]

    # Obtener las filas que están en dropped_df pero no en target_nan_df_non_nan_other_columns_df
    rows_to_keep = dropped_df.index.difference(target_nan_df_non_nan_other_columns_df.index)
    dropped_df_not_in_target_nan = dropped_df.loc[rows_to_keep]

    # Drop the rows with NaN values in 'target_station' from the original DataFrame
    remaining_df = df.dropna()
    
    # Train-test split (70% train, 30% test)
    train_df, test_df = train_test_split(remaining_df, test_size=0.3, random_state=42)

    # Initialize regression models
    linear_reg = LinearRegression()
    random_forest_reg = RandomForestRegressor()

    # Define performance metric (RMSE in this example)
    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))

    # Train models on the training set
    X_train = train_df.drop([target_station, 'Fecha'], axis=1)
    y_train = train_df[target_station]
    linear_reg.fit(X_train, y_train)
    random_forest_reg.fit(X_train, y_train)

    # Evaluate models on the test set
    X_test = test_df.drop([target_station, 'Fecha'], axis=1)
    y_test = test_df[target_station]
    linear_reg_rmse = root_mean_squared_error(y_test, linear_reg.predict(X_test))
    random_forest_rmse = root_mean_squared_error(y_test, random_forest_reg.predict(X_test))

    # Choose the best model based on performance (lower RMSE in this case)
    if linear_reg_rmse < random_forest_rmse:
        selected_model = linear_reg
        selected_model_name = 'Linear Regression'
        selected_model_rmse = linear_reg_rmse
    else:
        selected_model = random_forest_reg
        selected_model_name = 'Random Forest'
        selected_model_rmse = random_forest_rmse

    # Use the selected model to complete missing values in the target station
    feature_names = X_train.columns
    for index, row in target_nan_df_non_nan_other_columns_df.iterrows():
        if pd.isnull(row[target_station]):
            X_pred = row.drop([target_station, 'Fecha']).values.reshape(1, -1)
            X_pred_df = pd.DataFrame(X_pred, columns=feature_names)
            predicted_value = selected_model.predict(X_pred_df)[0]

            # Check if the predicted value is negative or close to 0
            if predicted_value < 0 or np.isclose(predicted_value, 0):
                predicted_value = 0
                
            target_nan_df_non_nan_other_columns_df.loc[index, target_station] = np.round(predicted_value,2)

    # Merge them back together
    merged_df = pd.concat([target_nan_df_non_nan_other_columns_df, remaining_df,dropped_df_not_in_target_nan], ignore_index=True)
    merged_df.sort_values(by='Fecha', inplace=True)
    return merged_df, selected_model_name, linear_reg_rmse,random_forest_rmse

# Cálculo de NaN y completar hasta que no haya valores faltantes
def complete_until_no_nan(df):
    nan_percentages = df.isna().mean() * 100  # Porcentaje de valores NaN
    ordered_columns = nan_percentages.sort_values().index.tolist()
    ordered_columns.pop(0)  # Eliminar la columna de 'Fecha'
    print(ordered_columns)

    resultados = {
        'Gauge': [],
        'Selected Model': []
    }

    previous_nan_sum = nan_percentages.sum()  # Sumar NaN antes de empezar

    # Iterar hasta que no queden valores faltantes
    while nan_percentages.sum() > 0:
        print(f"Porcentajes de NaN: \n{nan_percentages}")
        
        for gauge in ordered_columns:
            if df[gauge].isna().sum() > 0:  # Solo completar si la estación tiene valores faltantes
                # Completación inicial usando el mejor método
                df, model_name, linear_reg_rmse, random_forest_rmse = complete_missing_data(df, gauge)
                resultados['Gauge'].append(gauge)
                resultados['Selected Model'].append(model_name)
                print(f"Selected Model for {gauge}: {model_name}")

        # Recalcular los porcentajes de NaN después de completar los datos
        nan_percentages = df.isna().mean() * 100
        current_nan_sum = nan_percentages.sum()

        # Si no mejora el porcentaje de NaN, hacer regresión simple
        if current_nan_sum >= previous_nan_sum:
            print("No hubo mejora en los valores faltantes, buscando la mejor columna para completar...")
            for gauge in ordered_columns:
                if df[gauge].isna().sum() > 0:
                    # Identificar la mejor columna con menos NaN para hacer la regresión
                    best_column = identify_best_column_with_data(df.iloc[:, 1:], gauge)
                    print(f"Usando la columna {best_column} para completar {gauge}")
                    
                    # Hacer regresión simple usando la mejor columna
                    df = simple_regression(df, best_column, gauge)
                    print(f"Regresión simple completada para {gauge}")

        # Actualizamos el porcentaje de NaN para la siguiente iteración
        previous_nan_sum = current_nan_sum

    return df, pd.DataFrame(resultados)


In [2]:
obs=pd.read_csv('../data/Estaciones/gauge.csv',sep=',')
#obs['Fecha'] = pd.to_datetime(obs['Fecha'], dayfirst=True)
#obs['Fecha'] = pd.to_datetime(obs['Fecha'])
#obs.set_index('Fecha',inplace=True)
obs.head()

,Fecha,chusis,chalaco,huamarca,huancabamba,miraflores
0,1980-01-01,0.0,0.0,0.4,2.1,0.0
1,1980-01-02,0.0,0.0,0.0,0.0,0.0
2,1980-01-03,0.0,0.0,0.0,1.5,0.0
3,1980-01-04,0.0,0.0,0.0,0.0,0.0
4,1980-01-05,0.0,0.0,0.0,0.0,0.0


In [3]:
# Ejemplo de uso:
df_completado, resultados_df = complete_until_no_nan(obs)
# print(resultados_df)

['huancabamba', 'huamarca', 'chalaco', 'miraflores', 'chusis']
Porcentajes de NaN: 
Fecha           0.000000
chusis         17.406340
chalaco         4.178674
huamarca        3.940922
huancabamba     3.350144
miraflores      7.536023
dtype: float64
Selected Model for huancabamba: Linear Regression
Selected Model for huamarca: Linear Regression
Selected Model for chalaco: Linear Regression
Selected Model for miraflores: Linear Regression
Selected Model for chusis: Linear Regression
Porcentajes de NaN: 
Fecha          0.000000
chusis         6.563401
chalaco        1.988473
huamarca       2.672911
huancabamba    2.759366
miraflores     5.036023
dtype: float64
Selected Model for huancabamba: Linear Regression
Selected Model for huamarca: Linear Regression
Selected Model for chalaco: Linear Regression
Selected Model for miraflores: Random Forest
Selected Model for chusis: Linear Regression
No hubo mejora en los valores faltantes, buscando la mejor columna para completar...
Usando la column

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
resultados_df

In [ ]:
#obs['Fecha'] = pd.to_datetime(obs['Fecha'])
#obs.set_index('Fecha',inplace=True)
#plt.figure(figsize=(15,10))
#sns.heatmap(obs.isnull(), cbar=False)
#plt.show()

In [ ]:
#df_completado['Fecha'] = pd.to_datetime(df_completado['Fecha'])
#df_completado.set_index('Fecha',inplace=True)
#plt.figure(figsize=(15,10))
#sns.heatmap(df_completado.isnull(), cbar=False)
#plt.show()

In [22]:
def identify_best_column_with_data(df, target_station):
    """
    Identifica la mejor columna que tiene la menor cantidad de valores nulos
    en las filas donde la estación objetivo tiene valores faltantes.

    Parameters:
    - df: DataFrame que contiene las estaciones y los datos de las mismas.
    - target_station: La estación que tiene valores faltantes y que se quiere completar.

    Returns:
    - best_column: Nombre de la columna con la menor cantidad de valores nulos en las filas donde la estación objetivo tiene valores faltantes.
    """
    # Filtrar las filas donde la estación objetivo tiene valores NaN
    target_nan_df = df[df[target_station].isna()]

    # Verificar las columnas que tienen la menor cantidad de NaN en esas mismas filas
    missing_counts = target_nan_df.drop(columns=[target_station]).isna().sum()

    # Seleccionar la columna con la menor cantidad de NaN
    best_column = missing_counts.idxmin()

    return best_column

def simple_regression(df, best_column, target_station,linear_reg = None):
    # Filtrar las filas donde target_station tiene valores NaN
    target_nan_df = df[df[target_station].isna()]

    # Filtrar las filas donde best_column no tiene valores NaN (tiene datos)
    target_nan_df_non_nan_other_columns_df = target_nan_df[target_nan_df[best_column].notna()]

    if linear_reg == None:
        # Filtrar las filas completas (sin NaN) en 'best_column' y 'target_station' para entrenar el modelo
        remaining_df = df[['Fecha', best_column, target_station]].dropna()
    
        # Realizar Train-test split (70% train, 30% test)
        train_df, test_df = train_test_split(remaining_df, test_size=0.3, random_state=42)
    
        # Inicializar el modelo de regresión lineal
        linear_reg = LinearRegression()
    
        # Entrenar el modelo
        X_train = train_df[[best_column]]  # Usar solo 'best_column' como predictor
        y_train = train_df[target_station]
        linear_reg.fit(X_train, y_train)

    # Usar el modelo entrenado para completar los valores faltantes en target_station
    for index, row in target_nan_df_non_nan_other_columns_df.iterrows():
        # Crear un DataFrame con el valor de 'best_column' para mantener los nombres de las características
        X_pred_df = pd.DataFrame([[row[best_column]]], columns=[best_column])
        predicted_value = linear_reg.predict(X_pred_df)[0]

        # Verificar si el valor predicho es negativo o cercano a 0
        if predicted_value < 0 or np.isclose(predicted_value, 0):
            predicted_value = 0

        # Rellenar el valor predicho en target_station
        target_nan_df_non_nan_other_columns_df.loc[index, target_station] = np.round(predicted_value, 2)

    # Fusionar los datos completados con el resto del DataFrame
    merged_df = pd.concat([target_nan_df_non_nan_other_columns_df, remaining_df], ignore_index=True)
    merged_df.sort_values(by='Fecha', inplace=True)
    
    return merged_df,linear_reg
# Function to perform train-test split and complete missing data using regression
def complete_missing_data(df, target_station, selected_model=None):
    # Drop rows with NaN values in the target station
    # Create a copy of the dropped rows
    dropped_df = df[df.isna().any(axis=1)]

    # Filtrar las filas donde target_station es NaN
    target_nan_df = df[df[target_station].isna()]

    # Filtrar las filas donde el resto de las columnas no son NaN
    target_nan_df_non_nan_other_columns_df = target_nan_df[target_nan_df.drop(columns=[target_station]).notna().all(axis=1)]

    # Obtener las filas que están en dropped_df pero no en target_nan_df_non_nan_other_columns_df
    rows_to_keep = dropped_df.index.difference(target_nan_df_non_nan_other_columns_df.index)
    dropped_df_not_in_target_nan = dropped_df.loc[rows_to_keep]

    if selected_model is None:
        # Drop the rows with NaN values in 'target_station' from the original DataFrame
        remaining_df = df.dropna()
        
        # Verificar si hay suficientes datos para el train-test split
        if len(remaining_df) < 2:
            print(f"No hay suficientes datos para realizar regresión en la estación: {target_station}")
            return df, selected_model, None, None, None
        
        # Train-test split (70% train, 30% test)
        train_df, test_df = train_test_split(remaining_df, test_size=0.3, random_state=42)
    
        # Initialize regression models
        linear_reg = LinearRegression()
        random_forest_reg = RandomForestRegressor()
    
        # Define performance metric (RMSE in this example)
        def root_mean_squared_error(y_true, y_pred):
            return np.sqrt(mean_squared_error(y_true, y_pred))
    
        # Train models on the training set
        X_train = train_df.drop([target_station, 'Fecha'], axis=1)
        y_train = train_df[target_station]
        linear_reg.fit(X_train, y_train)
        random_forest_reg.fit(X_train, y_train)

        # Evaluate models on the test set
        X_test = test_df.drop([target_station, 'Fecha'], axis=1)
        y_test = test_df[target_station]
        linear_reg_rmse = root_mean_squared_error(y_test, linear_reg.predict(X_test))
        random_forest_rmse = root_mean_squared_error(y_test, random_forest_reg.predict(X_test))
    
        # Choose the best model based on performance (lower RMSE in this case)
        if linear_reg_rmse < random_forest_rmse:
            selected_model = linear_reg
            selected_model_name = 'Linear Regression'
            selected_model_rmse = linear_reg_rmse
        else:
            selected_model = random_forest_reg
            selected_model_name = 'Random Forest'
            selected_model_rmse = random_forest_rmse
    else:
        selected_model = selected_model
        selected_model_rmse = None

    # Use the selected model to complete missing values in the target station
    feature_names = [col for col in df.columns if col not in [target_station, 'Fecha']]
    for index, row in target_nan_df_non_nan_other_columns_df.iterrows():
        if pd.isnull(row[target_station]):
            X_pred = row.drop([target_station, 'Fecha']).values.reshape(1, -1)
            X_pred_df = pd.DataFrame(X_pred, columns=feature_names)
            predicted_value = selected_model.predict(X_pred_df)[0]

            # Check if the predicted value is negative or close to 0
            if predicted_value < 0 or np.isclose(predicted_value, 0):
                predicted_value = 0
                
            target_nan_df_non_nan_other_columns_df.loc[index, target_station] = np.round(predicted_value, 2)

    # Merge them back together
    merged_df = pd.concat([target_nan_df_non_nan_other_columns_df, remaining_df, dropped_df_not_in_target_nan], ignore_index=True)
    merged_df.sort_values(by='Fecha', inplace=True)
    return merged_df, selected_model, linear_reg_rmse, random_forest_rmse, selected_model_name


# Cálculo de NaN y completar hasta que no haya valores faltantes
def complete_until_no_nan(df):
    nan_percentages = df.isna().mean() * 100  # Porcentaje de valores NaN
    ordered_columns = nan_percentages.sort_values().index.tolist()
    ordered_columns.pop(0)  # Eliminar la columna de 'Fecha'
    print(ordered_columns)

    resultados = {
        'Gauge': [],
        'Selected Model': []
    }

    previous_nan_sum = nan_percentages.sum()  # Sumar NaN antes de empezar

    model_rg = None
    linear_reg = None
    # Iterar hasta que no queden valores faltantes
    while nan_percentages.sum() > 0:
        print(f"Porcentajes de NaN: \n{nan_percentages}")
        
        for gauge in ordered_columns:
            if df[gauge].isna().sum() > 0:  # Solo completar si la estación tiene valores faltantes
                # Completación inicial usando el mejor método
                df, model_rg, linear_reg_rmse, random_forest_rmse,model_name = complete_missing_data(df, gauge,selected_model = model_rg)
                print(model_rg)
                resultados['Gauge'].append(gauge)
                resultados['Selected Model'].append(model_name)
                print(f"Selected Model for {gauge}: {model_name}")

        # Recalcular los porcentajes de NaN después de completar los datos
        nan_percentages = df.isna().mean() * 100
        current_nan_sum = nan_percentages.sum()

        # Si no mejora el porcentaje de NaN, hacer regresión simple
        if current_nan_sum >= previous_nan_sum:
            print("No hubo mejora en los valores faltantes, buscando la mejor columna para completar...")
            for gauge in ordered_columns:
                if df[gauge].isna().sum() > 0:
                    # Identificar la mejor columna con menos NaN para hacer la regresión
                    best_column = identify_best_column_with_data(df.iloc[:, 1:], gauge)
                    print(f"Usando la columna {best_column} para completar {gauge}")
                    
                    # Hacer regresión simple usando la mejor columna
                    df, linear_reg = simple_regression(df, best_column, gauge,linear_reg = linear_reg)
                    print(f"Regresión simple completada para {gauge}")

        # Actualizamos el porcentaje de NaN para la siguiente iteración
        previous_nan_sum = current_nan_sum

    return df, pd.DataFrame(resultados)

In [23]:
# Ejemplo de uso:
df_completado, resultados_df = complete_until_no_nan(obs)
# print(resultados_df)

['huancabamba', 'huamarca', 'chalaco', 'miraflores', 'chusis']
Porcentajes de NaN: 
Fecha           0.000000
chusis         17.406340
chalaco         4.178674
huamarca        3.940922
huancabamba     3.350144
miraflores      7.536023
dtype: float64
LinearRegression()
Selected Model for huancabamba: Linear Regression


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- huancabamba
Feature names seen at fit time, yet now missing:
- huamarca
